In [5]:
"""
Generador de Gráficas Independientes - Emergencias Colombia 1998-2024
Ejecutar: python generar_graficas_independientes.py
Genera 10 archivos HTML en la carpeta de salida:
  g01_eventos_por_año.html
  g02_distribucion_tipo_evento.html
  g03_top15_departamentos.html
  g04_heatmap_mes_año.html
  g05_victimas_por_año.html
  g06_treemap_personas.html
  g07_victimas_por_departamento.html
  g08_estacionalidad_mensual.html
  g09_familias_vs_viviendas.html
  g10_personas_por_año.html
"""

import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import csv as _csv
import unicodedata
import os
import warnings
warnings.filterwarnings("ignore")

# ── CONFIGURACIÓN ─────────────────────────────────────────────────────────────
FILE      = r"C:\Users\User\Downloads\CONSOLIDADO_EMERGENCIAS_1998-2024_v2.csv"
OUTPUT_DIR = r"C:\Users\User\Downloads\graficas_emergencias"

os.makedirs(OUTPUT_DIR, exist_ok=True)

# ── CARGAR DATOS ──────────────────────────────────────────────────────────────
print("Cargando datos...")
with open(FILE, "rb") as f:
    muestra = f.read(4000).decode("latin-1")
sep = _csv.Sniffer().sniff(muestra, delimiters=",;\t|").delimiter
print(f"  → Separador detectado: '{sep}'")

for enc in ["utf-8-sig", "utf-8", "latin-1"]:
    try:
        df = pd.read_csv(FILE, sep=sep, dtype=str, encoding=enc, on_bad_lines="skip")
        df.columns = df.columns.str.strip()
        cols_str = " ".join(df.columns)
        if "AÑO" in cols_str or "ANO" in cols_str or "AÃ" not in cols_str:
            print(f"  → Encoding usado: {enc}")
            break
    except Exception as e:
        print(f"  → {enc} falló: {e}")

print(f"  → {len(df):,} filas cargadas, {len(df.columns)} columnas")

# ── LIMPIAR ───────────────────────────────────────────────────────────────────
def _norm(s):
    return unicodedata.normalize("NFKD", str(s)).encode("ascii","ignore").decode().strip().upper()

col_norm = {_norm(c): c for c in df.columns}

def _find(nombre):
    if nombre in df.columns: return nombre
    k = _norm(nombre)
    if k in col_norm: return col_norm[k]
    for orig_norm, orig in col_norm.items():
        if k in orig_norm or orig_norm in k:
            return orig
    return None

for estandar in ["AÑO", "MES", "DEPARTAMENTO", "MUNICIPIO", "EVENTO"]:
    real = _find(estandar)
    if real and real != estandar:
        df.rename(columns={real: estandar}, inplace=True)

NUM_COLS = ["MUERTOS", "HERIDOS", "DESAPA.", "PERSONAS", "FAMILIAS",
            "VIV.DESTRU.", "VIV.AVER.", "HECTAREAS", "RECURSOS EJECUTADOS"]
for c in NUM_COLS:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce").fillna(0)

for col in ["AÑO", "MES"]:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")

for col in ["DEPARTAMENTO", "MUNICIPIO", "EVENTO"]:
    if col in df.columns:
        df[col] = df[col].astype(str).str.strip().str.upper()

if "AÑO" not in df.columns:
    for c in df.columns:
        try:
            tmp = pd.to_numeric(df[c], errors="coerce")
            if tmp.between(1990, 2030).sum() > len(df) * 0.5:
                df.rename(columns={c: "AÑO"}, inplace=True)
                df["AÑO"] = tmp
                break
        except: pass

df = df[df["AÑO"].notna()].copy()
df["AÑO"] = df["AÑO"].astype(int)

MESES_NOMBRE = {1:"Ene",2:"Feb",3:"Mar",4:"Abr",5:"May",6:"Jun",
                7:"Jul",8:"Ago",9:"Sep",10:"Oct",11:"Nov",12:"Dic"}

# ── PALETA ────────────────────────────────────────────────────────────────────
DARK_BG = "#050d14"
PAPER   = "#07111a"
FONT_C  = "#d4e9f7"
GRID_C  = "#0e2233"
ACCENT1 = "#f5c518"
ACCENT2 = "#e63946"
ACCENT3 = "#00e5a0"
ACCENT4 = "#3a9eca"
TITLE_C = "#ffffff"

base_layout = dict(
    paper_bgcolor=PAPER, plot_bgcolor=DARK_BG,
    font=dict(color=FONT_C, size=12, family="'Segoe UI', Arial, sans-serif"),
    margin=dict(l=60, r=30, t=70, b=60),
    xaxis=dict(gridcolor=GRID_C, zerolinecolor=GRID_C, tickfont=dict(color=FONT_C)),
    yaxis=dict(gridcolor=GRID_C, zerolinecolor=GRID_C, tickfont=dict(color=FONT_C)),
    title_font=dict(color=TITLE_C, size=15, family="'Segoe UI', Arial, sans-serif"),
)

# ── PLANTILLA HTML ────────────────────────────────────────────────────────────
PLOTLY_CDN = "https://cdn.plot.ly/plotly-2.27.0.min.js"
GOOGLE_FONT = "https://fonts.googleapis.com/css2?family=Inter:wght@400;600;700;900&display=swap"

def wrap_html(fig, titulo, subtitulo=""):
    """Envuelve una figura Plotly en un HTML completo con el estilo oscuro."""
    fig_html = pio.to_html(
        fig, full_html=False, include_plotlyjs=False,
        config={"responsive": True, "displayModeBar": True,
                "modeBarButtonsToRemove": ["select2d", "lasso2d"],
                "displaylogo": False}
    )
    return f"""<!DOCTYPE html>
<html lang="es">
<head>
<meta charset="UTF-8">
<meta name="viewport" content="width=device-width, initial-scale=1.0">
<title>{titulo}</title>
<script src="{PLOTLY_CDN}"></script>
<link href="{GOOGLE_FONT}" rel="stylesheet">
<style>
  * {{ box-sizing: border-box; margin: 0; padding: 0; }}
  body {{
    font-family: 'Inter', 'Segoe UI', Arial, sans-serif;
    background: #020b12;
    color: #d4e9f7;
    min-height: 100vh;
    display: flex;
    flex-direction: column;
  }}
  .chart-container {{
    flex: 1;
    padding: 20px 28px;
    display: flex;
    flex-direction: column;
  }}
  .chart-wrapper {{
    flex: 1;
    background: #07111a;
    border: 1px solid #0e2233;
    border-radius: 6px;
    overflow: hidden;
    min-height: 520px;
  }}
  .chart-wrapper > div {{
    width: 100% !important;
    height: 100% !important;
    min-height: 520px;
  }}
  footer {{
    text-align: center;
    padding: 12px;
    color: #1a3a55;
    font-size: 0.65rem;
    letter-spacing: 1px;
    text-transform: uppercase;
    border-top: 1px solid #0a1e2e;
  }}
</style>
</head>
<body>
<div class="chart-container">
  <div class="chart-wrapper">
    {fig_html}
  </div>
</div>
<footer>Datos UNGRD Colombia · Generado con Python + Plotly</footer>
</body>
</html>"""

def guardar(fig, nombre_archivo, titulo, subtitulo=""):
    path = os.path.join(OUTPUT_DIR, nombre_archivo)
    with open(path, "w", encoding="utf-8") as f:
        f.write(wrap_html(fig, titulo, subtitulo))
    print(f"  ✅ {nombre_archivo}")


# ══════════════════════════════════════════════════════════════════════════════
print("\nGenerando gráficas...")

# ── G01: Área temporal ────────────────────────────────────────────────────────
by_year = df.groupby("AÑO").size().reset_index(name="eventos")
fig1 = px.area(by_year, x="AÑO", y="eventos",
               title="📈 Eventos Registrados por Año (1998–2024)",
               color_discrete_sequence=[ACCENT1])
fig1.update_layout(**base_layout)
fig1.update_traces(fillcolor="rgba(245,197,24,0.15)", line=dict(width=2.5, color=ACCENT1))
guardar(fig1, "g01_eventos_por_año.html", "Eventos por Año")

# ── G02: Donut tipo evento ────────────────────────────────────────────────────
ev_counts = df["EVENTO"].value_counts().head(10).reset_index()
ev_counts.columns = ["EVENTO", "n"]
fig2 = px.pie(ev_counts, names="EVENTO", values="n",
              title="🔵 Distribución por Tipo de Evento (Top 10)",
              color_discrete_sequence=[ACCENT1, ACCENT2, ACCENT3, ACCENT4,
                                       "#a78bfa","#fb8c00","#26c6da","#ef5350",
                                       "#66bb6a","#ab47bc"],
              hole=0.45)
fig2.update_layout(paper_bgcolor=PAPER, plot_bgcolor=DARK_BG,
                   font=dict(color=FONT_C, family="'Inter', Arial"),
                   margin=dict(l=20, r=20, t=70, b=20),
                   title_font=dict(color=TITLE_C, size=15))
guardar(fig2, "g02_distribucion_tipo_evento.html", "Distribución por Tipo de Evento")

# ── G03: Barras horizontales departamentos ────────────────────────────────────
dep_ev = df.groupby("DEPARTAMENTO").size().nlargest(15).reset_index(name="n").sort_values("n")
fig3 = px.bar(dep_ev, x="n", y="DEPARTAMENTO", orientation="h",
              title="🏔️ Top 15 Departamentos con más Eventos",
              color="n",
              color_continuous_scale=[[0,"#0e2233"],[0.4,"#1a6a9a"],[1.0,"#f5c518"]])
fig3.update_layout(**base_layout, coloraxis_showscale=False, yaxis_title="")
guardar(fig3, "g03_top15_departamentos.html", "Top 15 Departamentos")

# ── G04: Heatmap mes × año ────────────────────────────────────────────────────
hm = df.groupby(["AÑO","MES"]).size().reset_index(name="n")
hm_pivot = hm.pivot(index="MES", columns="AÑO", values="n").fillna(0)
hm_pivot.index = [MESES_NOMBRE.get(i, i) for i in hm_pivot.index]
fig4 = px.imshow(hm_pivot,
                 color_continuous_scale=[[0,"#050d14"],[0.3,"#0a3a5c"],
                                         [0.65,"#1a7abf"],[1.0,"#f5c518"]],
                 title="🔥 Heatmap de Eventos: Mes vs Año",
                 labels=dict(color="Eventos"), aspect="auto")
fig4.update_layout(paper_bgcolor=PAPER, font=dict(color=FONT_C, size=11),
                   margin=dict(l=60, r=30, t=70, b=50),
                   title_font=dict(color=TITLE_C, size=15))
guardar(fig4, "g04_heatmap_mes_año.html", "Heatmap Mes vs Año")

# ── G05: Víctimas apiladas por año ────────────────────────────────────────────
vcols = [c for c in ["MUERTOS","HERIDOS","DESAPA."] if c in df.columns]
vic_yr = df.groupby("AÑO")[vcols].sum().reset_index()
fig5 = go.Figure()
colors_vic = [ACCENT2, "#fb8c00", ACCENT1]
for i, col in enumerate(vcols):
    fig5.add_trace(go.Bar(x=vic_yr["AÑO"], y=vic_yr[col], name=col,
                          marker_color=colors_vic[i]))
fig5.update_layout(barmode="stack", title="💔 Víctimas por Año (Muertos · Heridos · Desaparecidos)",
                   **base_layout, legend=dict(bgcolor="rgba(0,0,0,0)", orientation="h", y=1.06))
guardar(fig5, "g05_victimas_por_año.html", "Víctimas por Año")

# ── G06: Treemap personas por departamento ────────────────────────────────────
tm = df.groupby("DEPARTAMENTO")["PERSONAS"].sum().nlargest(20).reset_index()
tm = tm[tm["PERSONAS"] > 0]
fig6 = px.treemap(tm, path=["DEPARTAMENTO"], values="PERSONAS",
                  title="🗺️ Personas Afectadas por Departamento (Top 20)",
                  color="PERSONAS",
                  color_continuous_scale=[[0,"#0a2a3e"],[0.5,"#1a6a9a"],[1.0,"#00e5a0"]])
fig6.update_layout(paper_bgcolor=PAPER, font=dict(color=FONT_C),
                   margin=dict(l=10, r=10, t=70, b=10),
                   title_font=dict(color=TITLE_C, size=15))
guardar(fig6, "g06_treemap_personas.html", "Treemap Personas Afectadas")

# ── G07: Barras apiladas horizontales — Víctimas por departamento ─────────────
vic_cols = [c for c in ["MUERTOS","HERIDOS","DESAPA."] if c in df.columns]
top10_dep = df["DEPARTAMENTO"].value_counts().head(10).index.tolist()
vic_dep = (df[df["DEPARTAMENTO"].isin(top10_dep)]
           .groupby("DEPARTAMENTO")[vic_cols].sum().reset_index())
vic_dep["TOTAL"] = vic_dep[vic_cols].sum(axis=1)
vic_dep = vic_dep.sort_values("TOTAL")

fig7 = go.Figure()
colors_v = {"MUERTOS": ACCENT2, "HERIDOS": "#fb8c00", "DESAPA.": ACCENT1}
for col in vic_cols:
    fig7.add_trace(go.Bar(
        name=col, x=vic_dep[col], y=vic_dep["DEPARTAMENTO"],
        orientation="h", marker_color=colors_v.get(col, "#95a5a6"),
        hovertemplate="<b>%{y}</b><br>" + col + ": %{x:,}<extra></extra>",
    ))
fig7.update_layout(
    barmode="stack",
    title="🩸 Composición de Víctimas — Top 10 Departamentos",
    paper_bgcolor=PAPER, plot_bgcolor=DARK_BG,
    font=dict(color=FONT_C, size=12, family="'Segoe UI', Arial, sans-serif"),
    margin=dict(l=10, r=30, t=70, b=50),
    xaxis=dict(title="Total Víctimas", gridcolor=GRID_C, zerolinecolor=GRID_C),
    yaxis=dict(title="", gridcolor=GRID_C, zerolinecolor=GRID_C),
    title_font=dict(color=TITLE_C, size=15),
    legend=dict(bgcolor="rgba(0,0,0,0)", orientation="h", y=1.08),
)
guardar(fig7, "g07_victimas_por_departamento.html", "Víctimas por Departamento")

# ── G08: Estacionalidad mensual ───────────────────────────────────────────────
est = df.groupby("MES").size().reset_index(name="n")
est["mes_nom"] = est["MES"].map(MESES_NOMBRE)
fig8 = px.bar(est, x="mes_nom", y="n",
              title="📅 Estacionalidad: Total de Eventos por Mes",
              color="n",
              color_continuous_scale=[[0,"#0a1e2e"],[0.5,"#1a6aaa"],[1.0,"#3a9eca"]])
fig8.update_layout(**base_layout, coloraxis_showscale=False, xaxis_title="Mes", yaxis_title="Total Eventos")
guardar(fig8, "g08_estacionalidad_mensual.html", "Estacionalidad Mensual")

# ── G09: Bubble Chart — Familias vs Viviendas ────────────────────────────────
bbl_agg = {"eventos": ("DEPARTAMENTO", "count")}
for c, alias in [("FAMILIAS","familias"),("VIV.DESTRU.","viv_dest"),
                 ("VIV.AVER.","viv_aver"),("PERSONAS","personas"),("MUERTOS","muertos")]:
    if c in df.columns:
        bbl_agg[alias] = (c, "sum")

bbl = df.groupby("DEPARTAMENTO").agg(**bbl_agg).reset_index()
for col in ["familias","viv_dest","viv_aver","personas","muertos"]:
    if col not in bbl.columns: bbl[col] = 0

bbl = bbl[bbl["familias"] > 0].nlargest(22, "personas").copy()
bbl["viv_total"] = bbl["viv_dest"] + bbl["viv_aver"]

fig9 = go.Figure()
fig9.add_trace(go.Scatter(
    x=bbl["familias"], y=bbl["viv_total"],
    mode="markers+text",
    text=bbl["DEPARTAMENTO"].str.title(),
    textposition="top center",
    textfont=dict(size=9, color=FONT_C),
    marker=dict(
        size=bbl["personas"] / bbl["personas"].max() * 80 + 10,
        color=bbl["muertos"],
        colorscale=[[0,"#0a3a5c"],[0.5,"#e63946"],[1.0,"#f5c518"]],
        showscale=True,
        colorbar=dict(
            title=dict(text="Fallecidos", font=dict(color=FONT_C, size=11)),
            tickfont=dict(color=FONT_C, size=10),
            len=0.6, thickness=14,
        ),
        opacity=0.85,
        line=dict(width=1, color="#0f1923"),
    ),
    hovertemplate=(
        "<b>%{text}</b><br>"
        "Familias afectadas: %{x:,}<br>"
        "Viviendas afectadas: %{y:,}<extra></extra>"
    ),
    showlegend=False,
))
fig9.update_layout(
    title="🫧 Familias vs Viviendas Afectadas por Departamento<br>"
          "<sup>Tamaño = personas afectadas · Color = fallecidos</sup>",
    paper_bgcolor=PAPER, plot_bgcolor=DARK_BG,
    font=dict(color=FONT_C, size=12, family="'Segoe UI', Arial, sans-serif"),
    margin=dict(l=60, r=90, t=80, b=60),
    xaxis=dict(title="Total Familias Afectadas", gridcolor=GRID_C, zerolinecolor=GRID_C),
    yaxis=dict(title="Total Viviendas Afectadas", gridcolor=GRID_C, zerolinecolor=GRID_C),
    title_font=dict(color=TITLE_C, size=15),
)
guardar(fig9, "g09_familias_vs_viviendas.html", "Familias vs Viviendas Afectadas")

# ── G10: Personas afectadas por año ──────────────────────────────────────────
pers_yr = df.groupby("AÑO")["PERSONAS"].sum().reset_index()
fig10 = px.line(pers_yr, x="AÑO", y="PERSONAS",
                title="👥 Total Personas Afectadas por Año",
                markers=True, color_discrete_sequence=[ACCENT3])
fig10.update_layout(**base_layout, yaxis_title="Total Personas Afectadas")
fig10.update_traces(line=dict(width=2.5), marker=dict(size=7, color=ACCENT3))
guardar(fig10, "g10_personas_por_año.html", "Personas Afectadas por Año")

# ── G00: Tarjeta de KPIs independiente ───────────────────────────────────────
total_ev      = len(df)
total_muertos = int(df["MUERTOS"].sum())  if "MUERTOS"    in df.columns else 0
total_pers    = int(df["PERSONAS"].sum()) if "PERSONAS"   in df.columns else 0
vd = df["VIV.DESTRU."].sum() if "VIV.DESTRU." in df.columns else 0
va = df["VIV.AVER."].sum()   if "VIV.AVER."   in df.columns else 0
total_viv     = int(vd + va)
deptos_afect  = df["DEPARTAMENTO"].nunique() if "DEPARTAMENTO" in df.columns else 0

kpi_page = f"""<!DOCTYPE html>
<html lang="es">
<head>
<meta charset="UTF-8">
<meta name="viewport" content="width=device-width, initial-scale=1.0">
<title>KPIs Emergencias Colombia 1998-2024</title>
<link href="https://fonts.googleapis.com/css2?family=Inter:wght@400;600;700;900&display=swap" rel="stylesheet">
<style>
  * {{ box-sizing: border-box; margin: 0; padding: 0; }}
  body {{
    font-family: 'Inter', 'Segoe UI', Arial, sans-serif;
    background: #020b12;
    color: #d4e9f7;
    min-height: 100vh;
    display: flex;
    flex-direction: column;
  }}
  .kpi-container {{
    flex: 1;
    display: flex;
    align-items: center;
    justify-content: center;
    padding: 60px 40px;
  }}
  .kpi-grid {{
    display: flex;
    flex-direction: row;
    flex-wrap: wrap;
    gap: 20px;
    width: 100%;
    justify-content: center;
  }}
  .kpi-card {{
    background: #07111a;
    border: 1px solid #0e2233;
    border-radius: 6px;
    padding: 28px 36px;
    display: flex;
    flex-direction: column;
    gap: 10px;
    flex: 1;
    min-width: 160px;
    max-width: 240px;
  }}
  .kpi-label {{
    font-size: 0.6rem;
    font-weight: 700;
    letter-spacing: 1.5px;
    color: #4a7a99;
    text-transform: uppercase;
    word-break: break-word;
    white-space: normal;
  }}
  .kpi-value {{
    font-size: 2.2rem;
    font-weight: 900;
    line-height: 1;
  }}
  footer {{
    text-align: center;
    padding: 14px;
    color: #1a3a55;
    font-size: 0.65rem;
    letter-spacing: 1px;
    text-transform: uppercase;
    border-top: 1px solid #0a1e2e;
  }}
</style>
</head>
<body>
<div class="kpi-container">
  <div class="kpi-grid">
    <div class="kpi-card">
      <div class="kpi-label">Eventos</div>
      <div class="kpi-value" style="color:#f5c518;">{total_ev:,}</div>
    </div>
    <div class="kpi-card">
      <div class="kpi-label">Fallecidos</div>
      <div class="kpi-value" style="color:#e63946;">{total_muertos:,}</div>
    </div>
    <div class="kpi-card">
      <div class="kpi-label">Personas Afectadas</div>
      <div class="kpi-value" style="color:#00e5a0;">{total_pers:,}</div>
    </div>
    <div class="kpi-card">
      <div class="kpi-label">Viviendas Afectadas</div>
      <div class="kpi-value" style="color:#3a9eca;">{total_viv:,}</div>
    </div>
    <div class="kpi-card">
      <div class="kpi-label">Dptos. Afectados</div>
      <div class="kpi-value" style="color:#a78bfa;">{deptos_afect:,}</div>
    </div>
  </div>
</div>
<footer>Datos UNGRD Colombia · Generado con Python + Plotly</footer>
</body>
</html>"""
path_kpi = os.path.join(OUTPUT_DIR, "g00_kpis.html")
with open(path_kpi, "w", encoding="utf-8") as f:
    f.write(kpi_page)
print("  ✅ g00_kpis.html")

# ─────────────────────────────────────────────────────────────────────────────
print(f"\n✅  11 archivos HTML generados en:")
print(f"    {OUTPUT_DIR}")
print("""
  g00_kpis.html
  g01_eventos_por_año.html
  g02_distribucion_tipo_evento.html
  g03_top15_departamentos.html
  g04_heatmap_mes_año.html
  g05_victimas_por_año.html
  g06_treemap_personas.html
  g07_victimas_por_departamento.html
  g08_estacionalidad_mensual.html
  g09_familias_vs_viviendas.html
  g10_personas_por_año.html
""")

Cargando datos...
  → Separador detectado: ';'
  → Encoding usado: utf-8-sig
  → 17,828 filas cargadas, 45 columnas

Generando gráficas...
  ✅ g01_eventos_por_año.html
  ✅ g02_distribucion_tipo_evento.html
  ✅ g03_top15_departamentos.html
  ✅ g04_heatmap_mes_año.html
  ✅ g05_victimas_por_año.html
  ✅ g06_treemap_personas.html
  ✅ g07_victimas_por_departamento.html
  ✅ g08_estacionalidad_mensual.html
  ✅ g09_familias_vs_viviendas.html
  ✅ g10_personas_por_año.html
  ✅ g00_kpis.html

✅  11 archivos HTML generados en:
    C:\Users\User\Downloads\graficas_emergencias

  g00_kpis.html
  g01_eventos_por_año.html
  g02_distribucion_tipo_evento.html
  g03_top15_departamentos.html
  g04_heatmap_mes_año.html
  g05_victimas_por_año.html
  g06_treemap_personas.html
  g07_victimas_por_departamento.html
  g08_estacionalidad_mensual.html
  g09_familias_vs_viviendas.html
  g10_personas_por_año.html

